# AUTOMATIC DIFFERENTIATION WITH `TORCH.AUTOGRAD`
Khi đào tạo mạng nơ-ron, thuật toán thường được sử dụng nhất là lan truyền ngược. Trong thuật toán này, các tham số (trọng số mô hình) được điều chỉnh theo độ dốc của hàm mất mát đối với tham số đã cho.

Để tính toán các độ dốc đó, PyTorch có một công cụ phân biệt tích hợp có tên là torch.autograd. Nó hỗ trợ tính toán tự động gradient cho bất kỳ đồ thị tính toán nào.

Hãy xem xét mạng nơ-ron một lớp đơn giản nhất, với đầu vào `x`, các tham số `w` và `b`, và một số hàm mất mát. Nó có thể được định nghĩa trong PyTorch theo cách sau:

In [1]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3) # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

## Tensor, chức năng và đồ thị tính toán 
Đoạn mã này xác định đồ thị tính toán sau:
![](comp-graph.png)

Trong mạng này, `w` và `b` là các tham số mà chúng ta cần tối ưu hóa. Do đó, chúng ta cần có khả năng tính toán các bậc của hàm mất mát liên quan đến các biến đó. Để làm điều đó, chúng tôi đặt thuộc tính `requi_grad` của các tensor đó.

>**Note** \
Bạn có thể đặt giá trị của `required_grad` khi tạo tensor hoặc sau này bằng cách sử dụng phương thức `x.requires_grad_(True)`.

Một hàm mà chúng ta áp dụng cho tensors để xây dựng đồ thị tính toán trên thực tế là một đối tượng của lớp `Function`. Đối tượng này biết cách tính hàm theo chiều thuận, và cũng biết cách tính đạo hàm của nó trong bước truyền ngược. Tham chiếu đến hàm truyền ngược được lưu trữ trong thuộc tính `grad_fn` của tensor. Bạn có thể tìm thêm thông tin của `Function` trong [tài liệu](https://pytorch.org/docs/stable/autograd.html#function).

In [2]:
print('Gradient function for z =',z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x00000255404E4BE0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x00000255404E4970>


## Tính Gradients
Để tối ưu hóa trọng số của các tham số trong mạng nơ-ron, chúng ta cần tính toán các dẫn xuất của hàm mất mát của chúng ta đối với các tham số, cụ thể là chúng ta cần $\frac{\partial loss}{\partial w} $ và $\frac{\partial loss}{\partial b}$ dưới một số giá trị cố định của `x` và `y`. Để tính toán các dẫn xuất đó, chúng tôi gọi `loss.backward()`, sau đó truy xuất các giá trị từ `w.grad` và `b.grad`:

In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0325, 0.0007, 0.0164],
        [0.0325, 0.0007, 0.0164],
        [0.0325, 0.0007, 0.0164],
        [0.0325, 0.0007, 0.0164],
        [0.0325, 0.0007, 0.0164]])
tensor([0.0325, 0.0007, 0.0164])


## Tắt theo dõi Gradient
Theo mặc định, tất cả các tensor có `requi_grad = True` đang theo dõi lịch sử tính toán của chúng và hỗ trợ tính toán gradient. Tuy nhiên, có một số trường hợp chúng ta không cần phải làm điều đó, ví dụ như khi chúng ta đã đào tạo mô hình và chỉ muốn áp dụng nó cho một số dữ liệu đầu vào, tức là chúng ta chỉ muốn thực hiện các phép tính chuyển tiếp thông qua mạng. Chúng tôi có thể ngừng theo dõi các phép tính bằng cách bao quanh mã tính toán của chúng tôi với khối `torch.no_grad()`:

In [4]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


Một cách khác để đạt được kết quả tương tự là sử dụng phương thức `detach()` trên tensor:

In [5]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


Có những lý do bạn có thể muốn tắt theo dõi gradient:
- Để đánh dấu một số tham số trong mạng nơ-ron của bạn tại các tham số được đóng băng (**frozen parameters**). Đây là một tình huống rất phổ biến để tinh chỉnh một mạng được đào tạo trước
- Để tăng tốc độ tính toán khi bạn chỉ thực hiện chuyển tiếp, bởi vì tính toán trên tensors không theo dõi gradient sẽ hiệu quả hơn.

## Thông tin thêm về Đồ thị tính toán
Về mặt khái niệm, autograd lưu giữ bản ghi dữ liệu (tensors) và tất cả các hoạt động đã thực thi (cùng với các tensor mới thu được) trong một đồ thị xoay chiều có hướng (**directed acyclic graph-DAG**) bao gồm các đối tượng [Funtion](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function). Trong DAG này, lá là tensor đầu vào, rễ là tensor đầu ra. Bằng cách theo dõi biểu đồ này từ gốc đến lá, bạn có thể tự động tính toán độ dốc bằng cách sử dụng quy tắc chuỗi.

Trong một lần chuyển tiếp, autograd thực hiện đồng thời hai việc:

- Chạy hoạt động được yêu cầu để tính toán kết quả một tensor.
- Duy trì chức năng gradient của hoạt động trong DAG. 

Đường lùi bắt đầu khi `.backward()` được gọi trên gốc DAG `.autograd` sau đó:

- Tính toán các gradient từ mỗi `.grad_fn`,
- Tích lũy chúng trong thuộc tính tensor’s `.grad` tương ứng
- Sử dụng quy tắc chuỗi, lan truyền tất cả các cách để tensor lá.

>**Note** \
Các DAG rất linh hoạt trong PyTorch Một điều quan trọng cần lưu ý là biểu đồ được tạo lại từ đầu; sau mỗi lần gọi `.backward()`, autograd bắt đầu điền vào một biểu đồ mới. Đây chính xác là những gì cho phép bạn sử dụng các câu lệnh luồng điều khiển trong mô hình của mình; bạn có thể thay đổi hình dạng, kích thước và hoạt động ở mỗi lần lặp nếu cần.

## Đọc tùy chọn: Tensor Gradients và các sản phẩm Jacobian
Trong nhiều trường hợp, chúng ta có một hàm mất mát vô hướng và chúng ta cần tính toán gradient đối với một số tham số. Tuy nhiên, có những trường hợp khi hàm đầu ra là một tensor tùy ý. Trong trường hợp này, PyTorch cho phép bạn tính cái gọi là **sản phẩm Jacobian**, chứ không phải gradient thực tế.

Đối với một hàm vectơ $\vec{y}=f(\vec{x})$ với $\vec{x}=(x_1, ..., x_n)$ và $\vec{y} = (y_1, ..., y_n)$ một gradient của $\vec{y}$ đối với $\vec{x}$ được cung cấp bởi ma trận Jacobian:\
![](Jacobian_matrix.png)

Thay vì tính toán ma trận Jacobian, PyTorch cho phép bạn tính toán Sản phẩm Jacobian $v^T.J$ cho một vectơ đầu vào nhất định $v = (v_1, ..., v_m)$. Điều này đạt được bằng cách gọi `backward` với $v$ làm đối số. Kích thước của $v$ phải giống với kích thước của tensor ban đầu, đối với mà chúng tôi muốn tính toán sản phẩm:

In [6]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


Lưu ý rằng khi chúng ta gọi `backward` lần thứ hai với cùng một đối số, giá trị của gradient sẽ khác. Điều này xảy ra bởi vì khi thực hiện truyền ngược, PyTorch **tích lũy các gradient**, tức là giá trị của các gradient được tính toán được thêm vào thuộc tính grad của tất cả các nút lá của đồ thị tính toán. Nếu bạn muốn tính toán các gradient thích hợp, bạn cần phải loại bỏ thuộc tính `grad` trước đó. Trong quá trình đào tạo thực tế, một trình tối ưu hóa sẽ giúp chúng tôi thực hiện điều này.

>**Note**\
Trước đây chúng ta đã gọi hàm `backward()` mà không có tham số. Điều này về cơ bản tương đương với việc gọi `backward(torch.tensor(1.0))`, đây là một cách hữu ích để tính toán độ dốc trong trường hợp hàm có giá trị vô hướng, chẳng hạn như mất mát trong quá trình đào tạo mạng nơ-ron.

## Further Readin
- [Autograd Mechanics](https://pytorch.org/docs/stable/notes/autograd.html)